# A RAMP Primer: Extracting URL Click Counts from a Subset of RAMP Data
---------------------------------------------------------------------------------------

## Requirements

The data needed to run this notebook include the monthly CSV subsets of RAMP data for 35 IR from January and Feburary, 2019. The data are available from Dryad at ...

**NOTE**: File paths in the code blocks below assume that the data and the *RAMP-IR-info.csv* needed to run the code are located in the same directory as this notebook. Please update paths or file locations as needed.

Please note the published data include additional data from March through May, 2019. The data files can be large, so this notebook only loads data from two months in order to conserve resources while still demonstrating the process of combining monthly CSV files of RAMP data.

This notebook was written using Python version 3, and Python Pandas is required:

1. Pandas [https://pandas.pydata.org/](https://pandas.pydata.org/)

The first code block below includes the pip commands to install Pandas. Uncomment and run as needed. Please note the RAMP team cannot provide support for configuring a Python or Jupyter environment.

In [1]:
# Uncomment the line below as needed to install dependencies.

# pip install --upgrade pandas

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

## Load the CSV File with Index Names

Data for each IR in RAMP are stored within an index specific to that IR. Since the published data include CSV dumps of the all RAMP for 35 participating IR, index names are needed in order to filter out data for a particular IR or subset of repositories. This information is provided in the file, *RAMP-IR-info.csv*, which is included in the GitHub repository for this notebook.

Complete documentation for RAMP data is available from the dataset record in Dryad.

In [3]:
# Read the CSV file.

repos = pd.read_csv('RAMP-IR-info.csv', encoding='latin-1')
print(repos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
ir_index_root           35 non-null object
ir_page_click_index     35 non-null object
ir_access_info_index    35 non-null object
dtypes: object(3)
memory usage: 920.0+ bytes
None


In [4]:
# Print information about RAMP participating IR.
# The 'ir_index_root' column is the information we need to filter for specific IR data.

repos.head()

,ir_index_root,ir_page_click_index,ir_access_info_index
0,swarthmore_inst-schol,swarthmore_inst-schol_page_clicks,swarthmore_inst-schol_access_info
1,caltech_authors,caltech_authors_page_clicks,caltech_authors_access_info
2,caltech_thesis,caltech_thesis_page_clicks,caltech_thesis_access_info
3,colorado,colorado_page_clicks,colorado_access_info
4,iupui_ir,iupui_ir_page_clicks,iupui_ir_access_info


## Read RAMP CSV Data

As stated above, the paths used in this notebook assume that the code and data are located within a single directory, and that the filenames are unchanged. Please update paths and filenames as needed.

The code in this notebook will:

1. Combine data from January and February, 2019.
2. Filter the combined data to get the count of citable content downloads (CCD) for the University of New Mexico Digital Repository. For a definition of CCD and an overview of the process used to determine CCD counts, please see available RAMP documentation at <https://osf.io/xbnmp/wiki/home/> or in the published dataset documentation.
3. Aggregate click counts on all pages - HTML pages as well as content files - by device and country of access.

Please note that citable content URLs for all participating IR, as defined in the documentation, are analyzed prior to indexing in RAMP. Differences in content URLs across IR platforms are managed as part of that process - within the included CSV data, there is no need to differentiate between DSpace, Digital Commons, ePrints, or other platforms. The steps outlined below can be used to analyze data for any RAMP IR.

In [5]:
# Read the CSVs - this will be slow when appending multiple large CSVs. 
# (We welcome contributions to improve this code! Please contact us to be added as a contributor.)

# In this example, we refer to the CSV files indvidually. A recommended
# alternative for loading several files is to use Python's 'os.walk' or 'glob'
# methods to read an arbitrary number of files.

# THIS BLOCK MAY TAKE A FEW MINUTES TO EXECUTE

# Create pointers to page level data.
janPageData = '2019-01_RAMP_subset_page-clicks.csv'
febPageData = '2019-02_RAMP_subset_page-clicks.csv'

# Read January page level data
rampPageData = pd.read_csv(janPageData)

# Append February page level data
rampPageData = rampPageData.append(pd.read_csv(febPageData))

# Get info - length, columns, etc.
print(rampPageData.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12812402 entries, 0 to 6156018
Data columns (total 8 columns):
position          float64
citableContent    object
clickThrough      float64
url               object
impressions       float64
date              object
clicks            float64
index             object
dtypes: float64(4), object(4)
memory usage: 879.8+ MB
None


In [6]:
# Inspect the first 5 rows

rampPageData.head()

,position,citableContent,clickThrough,url,impressions,date,clicks,index
0,19.0,Yes,0.0,https://scholarship.tricolib.brynmawr.edu/bitstream/handle/10066/10543/Nibi_44_528.pdf?sequence=1,2.0,2019-01-02,0.0,swarthmore_inst-schol_page_clicks
1,380.0,Yes,0.0,https://scholarship.tricolib.brynmawr.edu/bitstream/handle/10066/10553/Karras_44_530.pdf?sequence=1,1.0,2019-01-02,0.0,swarthmore_inst-schol_page_clicks
2,251.0,Yes,0.0,https://scholarship.tricolib.brynmawr.edu/bitstream/handle/10066/10592/Callanicos_44_533.pdf?sequence=1&isAllowed=y,1.0,2019-01-02,0.0,swarthmore_inst-schol_page_clicks
3,102.0,Yes,0.0,https://scholarship.tricolib.brynmawr.edu/bitstream/handle/10066/10598/Lewnes_44_533_B.pdf?sequence=1,1.0,2019-01-02,0.0,swarthmore_inst-schol_page_clicks
4,60.0,Yes,0.0,https://scholarship.tricolib.brynmawr.edu/bitstream/handle/10066/10611/Scofield_44_535.pdf?sequence=1,2.0,2019-01-02,0.0,swarthmore_inst-schol_page_clicks


In [7]:
# Filter for University of New Mexico Digital Repository data.
# From the RAMP index info printed above, the necessary index id is 'new_mexico_dc'.
# Note that the IR info file also includes page click and access info index names for each IR. As needed this info can be
# extracted/filtered dynamically. The process used here was chosen as requiring the least amount of code.

index = 'new_mexico_dc'

indexPageData = index + '_page_clicks'

irPageData = rampPageData[rampPageData['index'] == indexPageData].copy()

# Get info - length, columns, etc.
print(irPageData.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 467563 entries, 4519000 to 4360021
Data columns (total 8 columns):
position          467563 non-null float64
citableContent    467563 non-null object
clickThrough      467563 non-null float64
url               467563 non-null object
impressions       467563 non-null float64
date              467563 non-null object
clicks            467563 non-null float64
index             467563 non-null object
dtypes: float64(4), object(4)
memory usage: 32.1+ MB
None


In [8]:
# As above, inspect the first 5 rows.

irPageData.head()

,position,citableContent,clickThrough,url,impressions,date,clicks,index
4519000,255.5000,Yes,0.0,http://digitalrepository.unm.edu/cgi/viewcontent.cgi?article=1489&context=nma,2.0,2019-01-01,0.0,new_mexico_dc_page_clicks
4519001,21.0000,Yes,0.0,http://digitalrepository.unm.edu/cgi/viewcontent.cgi?article=1498&context=abya_yala,1.0,2019-01-01,0.0,new_mexico_dc_page_clicks
4519002,299.1875,Yes,0.0,http://digitalrepository.unm.edu/cgi/viewcontent.cgi?article=1500&context=abya_yala,16.0,2019-01-01,0.0,new_mexico_dc_page_clicks
4519003,87.0000,Yes,0.0,http://digitalrepository.unm.edu/cgi/viewcontent.cgi?article=1501&context=nma,2.0,2019-01-01,0.0,new_mexico_dc_page_clicks
4519004,150.0000,Yes,0.0,http://digitalrepository.unm.edu/cgi/viewcontent.cgi?article=1509&context=nma,7.0,2019-01-01,0.0,new_mexico_dc_page_clicks


### Summary Data: Row Counts and Citable Content Downloads

The following series of code blocks demonstrate the process described above for calculating or reproducing the metrics reported on the RAMP website. Each row in the dataframe represents data for a single page or URL as downloaded from Google Search Console on any given day. Metrics include:

1. A count of all rows in the dataset. This represents the amount of raw data downloaded from GSC.
2. A count of rows within which the URLs point to 'citable content' as defined in RAMP documentation.
3. A count of citable content downloads (CCD), as defined in RAMP documentation.

In [9]:
# Get count of rows returned. This is the same as the number of entries output by the info() method above.

print(len(irPageData))

467563


In [10]:
# Get count of rows with citable content
# To do this we are subsetting the data to only include rows where 'citableContent' is set to 'Yes.'

citableContentURLs = irPageData[irPageData['citableContent']=='Yes'].copy()
print(len(citableContentURLs))

396672


In [11]:
# CCD = the sum of clicks on citable content URLs
# Using our example arguments, the output of this code block equals the number of citable content downloads
#       from UNM's Digital Commons repository between June 1 and July 31, 2018.

print(citableContentURLs['clicks'].sum())

107513.0


## Sum of Clicks by Device and Country

Device type and country of access data must be processed separately from URL specific, page-level data. Since URLs are not provided with country and device type data, it is not possible to filter out citable content, and so click counts as provided below represent activity on all of the pages in an IR and not just content pages (PDFs, CSVs, etc.)

The RAMP web application also provides information about access by device and country. The next few code blocks reproduce those aggregations.

In [12]:
# Read the CSV data, similar to above.

# Create poiners to country/device access data.
janAccessData = '2019-01_RAMP_subset_country-device-info.csv'
febAccessData = '2019-02_RAMP_subset_country-device-info.csv'

# Read January country/device access data
rampAccessData = pd.read_csv(janAccessData)

# Append February country/device access data
rampAccessData = rampAccessData.append(pd.read_csv(febAccessData))

# Get info - length, columns, etc.
print(rampAccessData.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1749965 entries, 0 to 839728
Data columns (total 8 columns):
position        float64
clickThrough    float64
country         object
device          object
impressions     float64
date            object
clicks          float64
index           object
dtypes: float64(4), object(4)
memory usage: 120.2+ MB
None


In [13]:
# Inspect the first 5 rows.

rampAccessData.head()

,position,clickThrough,country,device,impressions,date,clicks,index
0,79.500000,0.000000,pry,DESKTOP,2.0,2019-01-25,0.0,swarthmore_inst-schol_access_info
1,57.376068,0.000000,idn,DESKTOP,117.0,2019-01-28,0.0,swarthmore_inst-schol_access_info
2,46.794118,0.029412,ukr,DESKTOP,34.0,2019-01-30,1.0,swarthmore_inst-schol_access_info
3,23.394089,0.034483,gbr,DESKTOP,203.0,2019-01-27,7.0,swarthmore_inst-schol_access_info
4,9.600000,0.000000,mar,MOBILE,10.0,2019-01-31,0.0,swarthmore_inst-schol_access_info


In [14]:
# Filter for University of New Mexico Digital Repository data.
# From the RAMP index info printed above, the necessary index id is 'new_mexico_dc'.
# Same as above, please note that the IR info file also includes page click and access info index names for each IR. 
# As needed this info can be extracted/filtered dynamically. 
# The process used here was chosen as requiring the least amount of code.

index = 'new_mexico_dc'

indexAccessData = index + '_access_info'

irAccessData = rampAccessData[rampAccessData['index'] == indexAccessData].copy()

# Get info - length, columns, etc.
print(irAccessData.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69365 entries, 567046 to 556031
Data columns (total 8 columns):
position        69365 non-null float64
clickThrough    69365 non-null float64
country         69365 non-null object
device          69365 non-null object
impressions     69365 non-null float64
date            69365 non-null object
clicks          69365 non-null float64
index           69365 non-null object
dtypes: float64(4), object(4)
memory usage: 4.8+ MB
None


In [15]:
# Inspect the first 5 rows.

irAccessData.head()

,position,clickThrough,country,device,impressions,date,clicks,index
567046,153.750289,0.001539,usa,DESKTOP,2599.0,2019-01-01,4.0,new_mexico_dc_access_info
567047,135.505515,0.001838,ecu,DESKTOP,544.0,2019-01-01,1.0,new_mexico_dc_access_info
567048,239.000000,0.000000,alb,DESKTOP,2.0,2019-01-01,0.0,new_mexico_dc_access_info
567049,190.000000,0.000000,arg,MOBILE,1.0,2019-01-01,0.0,new_mexico_dc_access_info
567050,37.000000,0.000000,aus,TABLET,1.0,2019-01-01,0.0,new_mexico_dc_access_info


In [16]:
# Sum of clicks by Device

clicksByDevice = irAccessData.groupby(['device'])['clicks'].sum()
print(clicksByDevice)

device
DESKTOP    84701.0
MOBILE     33704.0
TABLET     3107.0 
Name: clicks, dtype: float64


In [17]:
# Sum of clicks by Country
# The output is a long list - to see the whole list, replace 'clicksByCountry.head()' with 'clicksByCountry'.

clicksByCountry = irAccessData.groupby(['country'], sort=True)['clicks'].sum()
clicksByCountry.head(10)

country
abw    4.0   
afg    18.0  
ago    9.0   
aia    0.0   
ala    0.0   
alb    28.0  
and    3.0   
are    177.0 
arg    2346.0
arm    28.0  
Name: clicks, dtype: float64

In [18]:
# The default sort on the country access data is alphabetical by country code.
# Use the following to sort by click counts, showing top 10 countries by CCD.

clicksByCountry.sort_values(ascending=False).head(10)

country
usa    42753.0
ecu    13612.0
mex    10469.0
col    5260.0 
ind    4155.0 
gbr    3746.0 
per    2604.0 
esp    2499.0 
can    2376.0 
arg    2346.0 
Name: clicks, dtype: float64

In the examples as given, data for all dates within the dataset are aggregated into a single result.

Depending on the available date range, it may also be useful to group data by date AND device, or by date AND country. Example code for this is provided below.

In [47]:
# Sum of clicks by Date and Device
# The output is long - here it is limited to the first 10 lines.

clicksByDateDevice = irAccessData.groupby(['date', 'device'])['clicks'].sum()
print(clicksByDateDevice.head(10))

date        device 
2019-01-01  DESKTOP    589.0 
            MOBILE     299.0 
            TABLET     33.0  
2019-01-02  DESKTOP    1015.0
            MOBILE     431.0 
            TABLET     49.0  
2019-01-03  DESKTOP    1187.0
            MOBILE     444.0 
            TABLET     29.0  
2019-01-04  DESKTOP    983.0 
Name: clicks, dtype: float64


In [48]:
# Sum of clicks by Date and Country
# This output is also long and limited here to the first 20 lines.

clicksByDateCountry = irAccessData.groupby(['date', 'country'])['clicks'].sum()
clicksByDateCountry.head(20).sort_values(ascending=False)

date        country
2019-01-01  arg        15.0
            aus        11.0
            bel        4.0 
            bgd        2.0 
            alb        2.0 
            afg        1.0 
            aze        1.0 
            aut        1.0 
            bgr        1.0 
            ago        0.0 
            and        0.0 
            are        0.0 
            bhr        0.0 
            arm        0.0 
            atg        0.0 
            bdi        0.0 
            ben        0.0 
            bes        0.0 
            bfa        0.0 
            abw        0.0 
Name: clicks, dtype: float64

## Output to File

There may be some interest in saving some of this information to file. To save as different aggregations and summaries in CSV format, uncomment the desired outputs and run the code below.

In [49]:
# Save to CSV - uncomment specific lines to save those data to CSV format.
# Files will be saved to the directory this notebook is in.

# Using our example argument from above, the saved page-click data file will be named 'new_mexico_dc_RAMP_page-clicks.csv.'

# Uncomment the next line to save all of the raw data returned from ES.  Note that this could be a big file.
# irPageData.to_csv(index+'_RAMP_page-clicks.csv', index=False)

# Uncomment the next line to save rows with citable content to CSV. The file name follows a similar convention as above.
# citableContentURLs.to_csv(index+'_RAMP_citable_content.csv', in# dex=False)

# Uncomment the next line to save the clicks by Device info.
# clicksByDevice.to_csv(index+'_RAMP_clicks_device.csv')

# Uncomment the next line to save the clicks by Country info.
# clicksByCountry.to_csv(index+'_RAMP_clicks_country.csv')